In [160]:
import pandas as pd
import datetime as dt
import regex as re
import os
from pathlib import Path
import utils_laramie
import numpy as np

### Read in the data

In [161]:
# def get_all_raw_data(dirpath = Path('raw_data/')):
#     csv_list = [filename for filename in os.listdir(dirpath)]
#     return csv_list
# def clean_col_names(list_dfs, list_csvs):

#     list_cols = [df.columns.to_list() for df in list_dfs]
#     zipped = zip(list_csvs,list_cols)
#     name_list = []
#     column_list = []
#     for tup in list(zipped):
#         name = tup[0]
#         match = re.match(r'(\w+)_',name)
#         name = match.group(1)
#         for col in tup[1]:
#             if name.lower() != col.lower():
#                 name_list.append((name.upper()+'_'+col.lower()))
#             else:
#                 name_list.append(name.upper())

#     return name_list

# def drop_unnamed(df):
#     for col in df.columns:
#         if 'Unnamed' in str(col):
#             df.drop(columns= col, inplace=True)
#     return df
    
# def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
#     """concat a list of csvs into a single df"""
#     # make a list of dataframes
#     list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
#     for df in list_dfs:
#         df.drop_duplicates(inplace=True)
#         df['date'] = pd.to_datetime(df['date']).dt.date
#         df['date'] = pd.to_datetime(df['date'])
#         df = drop_unnamed(df)
#     clean_headers = clean_col_names(list_dfs,list_of_csvs)
#     merged_df = pd.concat(list_dfs, axis=1, join= 'inner')
#     merged_df.columns = clean_headers
#     return merged_df
# x = get_df()
# x 

In [162]:
# def get_df(list_of_csvs= ['SPY_data.csv','TR_data.csv','VIX_data.csv']):
#     """concat a list of csvs into a single df"""
#     list_dfs = [pd.read_csv(f'raw_data/{_csv}', parse_dates = True, infer_datetime_format = True) for _csv in list_of_csvs]
#     for df in list_dfs:
#         df.drop_duplicates(inplace=True)
#         df['date'] = pd.to_datetime(df['date']).dt.date
#         df['date'] = pd.to_datetime(df['date'])
#         df = drop_unnamed(df)
#         df.set_index('date',inplace=True)
#     clean_headers = clean_col_names(list_dfs,list_of_csvs)
#     merged_df = pd.concat(list_dfs, axis=1, join= 'inner')  
#     merged_df.columns = clean_headers
#     return merged_df
# x = get_df()
# x 

In [163]:
# imports for creating dataframe
from utils_laramie import get_df, get_all_raw_data

# imports for getting weekly range
from utils_laramie import calc_weekly_range

#imports for grouping data into weekly windows
from utils_laramie import grp_y_wk_d, drop_off_weeks

#shape data
from utils_laramie import get_X_y




### Create Weekly Range

In [164]:
# def get_day_names(df):    
#     df = get_df(get_all_raw_data())
#     df.reset_index(inplace=True)
#     df['DayOfWeek'] = df['date'].dt.day_name()
#     df.set_index('date', inplace=True)
#     return df

# def calc_weekly_range(df):
#     """ Must run "get_day_names" first"""
#     week_high = 0  
#     week_low = 9999999
#     for index, row in df.iterrows():
#         if df.loc[index, 'DayOfWeek'] == 'Monday':
#             week_high = df.loc[index,'SPY_high']
#             week_low = df.loc[index,'SPY_low']
#         else: 
#             if df.loc[index,'SPY_high'] > week_high:
#                 week_high=df.loc[index,'SPY_high']
#             if df.loc[index,'SPY_low'] < week_low:
#                 week_low=df.loc[index,'SPY_low']
#             if df.loc[index,'DayOfWeek'] == 'Friday':
#                 df.loc[index,'weekly_range'] = week_high - week_low
#     return df.fillna(0)

In [165]:
df = (get_df(get_all_raw_data()))
df = calc_weekly_range(df)
df = grp_y_wk_d(df)
df = drop_off_weeks(df)
df.head()

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
0,209.20,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000
1,206.48,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000
2,206.20,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000
3,205.44,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000
4,203.38,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195


In [166]:
raw_cols = df.columns.to_list()

In [167]:
raw_cols

['SPY_open',
 'SPY_high',
 'SPY_low',
 'SPY_close',
 'SPY_volume',
 'SPY_trade_count',
 'SPY_vwap',
 'TR',
 'VIX_open',
 'VIX_high',
 'VIX_low',
 'VIX_close',
 'weekly_range']

In [168]:
df['day_range'] = df['TR']
df.insert(0, 'day_range', df.pop('day_range'))

df['pct_change'] = (df['SPY_close']-df['SPY_open'])/df['SPY_open']
df.insert(1, 'pct_change', df.pop('pct_change'))

df['day_range_VIX'] = df['VIX_high']-df['VIX_low']
df.insert(2, 'day_range', df.pop('day_range'))

df['pct_changeVIX'] = (df['VIX_close']-df['VIX_open'])/df['SPY_open']
df.insert(3, 'pct_changeVIX', df.pop('pct_changeVIX'))

df['mean_volume'] = df['SPY_volume']/df['SPY_trade_count']
df.insert(4, 'mean_volume', df.pop('mean_volume'))


In [169]:
df.head()

,pct_change,SPY_open,day_range,pct_changeVIX,mean_volume,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range,day_range_VIX
0,-0.004446,209.20,2.5295,0.000908,272.286495,209.7295,207.20,208.27,102027111.0,374705.0,208.276128,2.5295,15.65,17.18,15.58,15.84,0.0000,1.60
1,0.002470,206.48,2.5090,-0.000436,266.573402,208.2890,205.78,206.99,103372367.0,387782.0,206.966276,2.5090,17.69,18.33,16.52,17.60,0.0000,1.81
2,-0.004219,206.20,4.5000,0.007565,277.036449,208.6800,204.18,205.33,162401537.0,586210.0,206.034646,4.5000,18.05,20.13,15.72,19.61,0.0000,4.41
3,0.002044,205.44,2.2900,0.000438,286.743585,207.4300,205.14,205.86,116128858.0,404992.0,206.102975,2.2900,19.25,19.72,18.13,19.34,0.0000,1.59
4,-0.007375,203.38,4.3500,0.014898,315.219794,204.1400,201.51,201.88,211173305.0,669924.0,203.150102,4.3500,21.36,25.27,20.88,24.39,8.2195,4.39


In [170]:
df_clean = df.drop(columns=raw_cols[:-1])
df_clean = df[[c for c in df_clean if c not in ['weekly_range'] ] + ['weekly_range']]

In [173]:
df_clean.to_csv('clean_data/basic_spy_vix.csv')

In [89]:
X , y = get_X_y(df_clean)

c:\Users\Laramie\Desktop\Project_Two\Project2\utils_laramie.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target']= df['weekly_range'].shift(-5)
c:\Users\Laramie\anaconda3\envs\dev\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
c:\Users\Laramie\anaconda3\envs\dev\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [90]:
len(X)

210

In [91]:
len(y)

210

In [95]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D

In [96]:
class MinMaxScaler3D(MinMaxScaler):
    def fit_transform(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)
        
    # def inverse_transform(self, X):
    #     x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
    #     return np.reshape(super().inverse_transform(x), newshape=X.shape)


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [98]:
scaler = MinMaxScaler3D()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [115]:
display(X_train_scaled.shape)
display(y_train.shape)

(157, 5, 5)

(157, 1, 1)

In [134]:
model = Sequential()
model.add(Dense(100, activation='relu', batch_input_shape=(None, None, X_train_scaled.shape[-1]) ))
model.add(Dense(1, activation='relu'))


In [135]:
model.output_shape

(None, None, 1)

In [101]:
# from tensorflow.keras.layers import Input
# from tensorflow.keras.models import Model
# visible = Input(shape=X_train_scaled.shape[1:])
# hidden = Dense(X_train_scaled.shape[-1])(visible)
# model = Model(inputs=visible, outputs=hidden)


In [136]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

In [142]:
model_shallow = model.fit(X_train_scaled, y_train, validation_split=0.3, epochs=400)

Epoch 1/400
4/4 [==============================] - 0s 18ms/step - loss: 17.8139 - mse: 17.8139 - val_loss: 64.0857 - val_mse: 64.0857
Epoch 2/400
4/4 [==============================] - 0s 10ms/step - loss: 17.7815 - mse: 17.7815 - val_loss: 64.0006 - val_mse: 64.0006
Epoch 3/400
4/4 [==============================] - 0s 7ms/step - loss: 17.7531 - mse: 17.7532 - val_loss: 63.9103 - val_mse: 63.9103
Epoch 4/400
4/4 [==============================] - 0s 9ms/step - loss: 17.7301 - mse: 17.7301 - val_loss: 63.7602 - val_mse: 63.7602
Epoch 5/400
4/4 [==============================] - 0s 12ms/step - loss: 17.7016 - mse: 17.7016 - val_loss: 63.6658 - val_mse: 63.6658
Epoch 6/400
4/4 [==============================] - 0s 13ms/step - loss: 17.6791 - mse: 17.6791 - val_loss: 63.5843 - val_mse: 63.5843
Epoch 7/400
4/4 [==============================] - 0s 10ms/step - loss: 17.6599 - mse: 17.6599 - val_loss: 63.4757 - val_mse: 63.4757
Epoch 8/400
4/4 [==============================] - 0s 8ms/step -

In [143]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

2/2 - 0s - loss: 17.4066 - mse: 17.4066 - 15ms/epoch - 7ms/step


In [139]:
predicted = model.predict(X_test_scaled)

In [140]:
predicted.shape

(53, 5, 1)

In [141]:
predicted

array([[[ 5.007126 ],
        [ 5.8333597],
        [ 4.679219 ],
        [ 6.259147 ],
        [ 4.568589 ]],

       [[16.24184  ],
        [14.041265 ],
        [ 9.727432 ],
        [ 8.579732 ],
        [10.41443  ]],

       [[ 6.3503647],
        [ 6.7845354],
        [ 5.7635455],
        [ 5.389523 ],
        [ 5.468379 ]],

       [[10.856031 ],
        [13.139171 ],
        [16.088268 ],
        [13.665759 ],
        [13.393655 ]],

       [[ 5.200884 ],
        [ 5.0527163],
        [ 6.681947 ],
        [ 7.337763 ],
        [ 5.1569633]],

       [[ 6.943101 ],
        [ 6.557103 ],
        [ 5.780925 ],
        [ 6.352924 ],
        [ 8.496882 ]],

       [[ 6.80952  ],
        [ 6.568929 ],
        [ 6.104815 ],
        [ 6.177959 ],
        [ 4.0821276]],

       [[15.491928 ],
        [ 9.099268 ],
        [17.500036 ],
        [15.372403 ],
        [13.18666  ]],

       [[15.337645 ],
        [13.824668 ],
        [10.629412 ],
        [13.8144245],
        [12.6247

In [129]:
y_scaler = MinMaxScaler()
y_scaler.fit(y_train.reshape(-1,1))


MinMaxScaler()

In [130]:
predicted = y_scaler.inverse_transform(predicted.reshape(-1,1))

In [131]:
len(predicted)

265

In [147]:
y_test.flatten()

array([ 5.24  ,  5.81  ,  2.75  , 16.6027,  3.23  ,  5.4001,  3.67  ,
       24.95  , 18.31  , 13.87  , 18.5   ,  5.43  ,  3.2363,  1.92  ,
        3.24  ,  6.99  , 14.23  ,  2.17  ,  2.38  ,  2.495 ,  1.27  ,
        7.01  ,  8.0752,  4.42  ,  3.48  , 18.9   ,  5.265 , 13.82  ,
        3.61  ,  6.27  ,  3.83  ,  7.47  ,  6.15  ,  2.47  ,  4.0625,
        9.87  ,  4.5   ,  3.65  ,  5.49  , 11.04  ,  2.79  , 29.07  ,
        9.45  , 15.08  ,  6.99  ,  3.43  ,  3.34  ,  8.21  ,  5.43  ,
       12.45  , 11.76  ,  4.63  , 11.9   ])